# Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import joblib
import time
from flair.data import Sentence
from flair.data import Token
from flair.embeddings import WordEmbeddings
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
%matplotlib inline

device = torch.device("cuda:0")
dtype = torch.float

# embedding size of fasttext models
d_model = 300 

# select language
language_in = 'de'
language_out = 'en' 

# load word embedding models
embedding_model_en = WordEmbeddings('en')
embedding_model_de = WordEmbeddings('de')

## Hyperparameters

In [ ]:
path = ""
vocab_in = joblib.load(path+f'vocab_{language_in}.data')
vocab_out = joblib.load(path+f'vocab_{language_out}.data')

seq_len_encoder = vocab_in["max_sentence_len"] + 1
seq_len_decoder = vocab_out["max_sentence_len"] + 1
vocab_size = vocab_out["vocab_size"] + 1 # + <EOS>

In [ ]:
transformer = torch.load(path+f'transformer.pt')
transformer.eval()
print("model weights:",sum(p.numel() for p in transformer.parameters() if p.requires_grad))

In [ ]:
sine_mask = [2*i for i in range(d_model//2)]
cosine_mask = [2*i+1 for i in range(d_model//2)]
i = torch.tensor([i for i in range(d_model//2)]).to(device).to(dtype)

# pre compute positional encodings for 50 tokens
positional_encodings = []
for position in range(50):
    
    sine = torch.sin( position / ( 10000 ** (2*i/d_model) ) )
    cosine = torch.cos( position / ( 10000 ** (2*i/d_model) ) )
    
    position_enc = torch.zeros(d_model).to(device)
    position_enc[sine_mask] = sine
    position_enc[cosine_mask] = cosine

    positional_encodings.append(position_enc)

# return pre computed pos encoding
def positional_encoding(sentence):
    num_tokens = len(sentence)
    return positional_encodings[0:num_tokens]

In [ ]:
def translate(sentence):
    # constructed sentence
    sentence_translated = ""

    sentence_de = Sentence(sentence,language_code='de')
    # add <EOS> token
    sentence_de.add_token("<EOS>")
    # embed
    embedding_model_de.embed(sentence_de)
    # pos encoding
    pos_enc = positional_encoding(sentence_de)
    # add embedding and pos enc
    emb_in = torch.stack([token.embedding + pos_enc[i] for i,token in enumerate(sentence_de)])
    # add padding
    seq_len_in = emb_in.shape[0]
    padding_length_in = vocab_in["max_sentence_len"] + 1 - seq_len_in            
    padding = torch.zeros(padding_length_in, d_model).to(device)

    x_in = torch.unsqueeze(torch.cat([emb_in, padding], dim=0),dim=0)

    # starting point of ouput
    sentence_en = Sentence(' ')
    sentence_en.add_token('<SOS>')

    done = False
    i= 0
    while not done and i < vocab_out["max_sentence_len"] + 1:
        #print(sentence_en)

        # embed
        embedding_model_en.embed(sentence_en)
        # pos encoding
        pos_enc = positional_encoding(sentence_en)
        # add embedding and pos enc
        emb_out = torch.stack([token.embedding + pos_enc[i] for i,token in enumerate(sentence_en)])
        # add padding
        seq_len_out = emb_out.shape[0]
        padding_length_out = vocab_out["max_sentence_len"] + 1 - seq_len_out            
        padding = torch.zeros(padding_length_out, d_model).to(device)
        x_out = torch.unsqueeze(torch.cat([emb_out, padding], dim=0),dim=0)
        out = transformer(x_in, x_out)

        # reshape to batch_size, seq, vocab
        out = torch.reshape(out, (1, seq_len_decoder,vocab_size))
        out = F.softmax(out, dim=2) 
        max_index = torch.argmax(out[0], dim=1).cpu().numpy()
        # get relevant position
        token_index = max_index[i]

        token = vocab_out[token_index]
        if token == "<EOS>":
            done = True
        else:
            sentence_translated += token + " "
            sentence_en.add_token(token)

        i += 1

    return sentence_translated

In [ ]:
train_sentences_german = ['Ich habe dich einst gewarnt, aber du hast nicht hören wollen.',
                          'Ich freue mich, dass ausnahmsweise mal jemand meiner Meinung ist.',
                          'Ich kann dir das Geld, das du mir geliehen hast, nicht zurückzahlen.',
                          'Das Essen war schrecklich, aber ich habe mich nicht beklagt.',
                          'Diese Behauptungen entbehren einer wissenschaftlichen Grundlage.',
                          'Was hältst du davon, wenn wir heute Abend zur Abwechslung mal draußen essen?',
                          'Ich habe von ihm gehört, aber ich kenne ihn nicht persönlich.',
                          'Ich glaube, wir haben uns den Falschen für die Stelle gesucht.',
                          'Ich möchte glauben, dass wir aus unseren Fehlern lernen.',
                          'In Kalifornien haben die meisten Häuser einen Rohbau aus Holz.',
                          'Die Preise sind doppelt so hoch wie vor zehn Jahren']

for sentence in train_sentences_german:
    print(f'In: {sentence} | Out: {translate(sentence)}')

In [ ]:
# test sentences
test_sentences_german = ['Ich bin doppelt so alt wie vor zehn Jahren.', 'Ich bin sehr begeistert von deinem Talent.', 'Das ist ein Problem, das wir lösen können.',
                        'Sie genießen den sanften Wind am Meer.', 'Das ist wirklich das beste Eis der Stadt.',
                        'Wie viele Menschen gibt es auf der Welt?','Mir fällt kein neuer Satz ein.',
                        'Mein neues Modell ist ganz gut, obwohl es Schwierigkeiten gab.',
                        'Dieses Weihnachten ist wie kein anderes zuvor.','Wie viele Häuser stehen auf der anderen Straßenseite?',
                        'Morgen ist schon wieder Montag.','Das Fahrrad hat zwei Räder und eine Klingel und ist schnell.',
                        'Augen auf, sonst fällst du auf den Boden!', 'Die Polizei verhaftet den Verbrecher und setzt ihn in das Auto.',
                        'Ich habe zu wenig Sätze zum Trainieren des Modells.','Lasst die Frau auf dem Auto tanzen.',
                        'Es gibt so viele Möglichkeiten, aber ich bin nicht kreativ genug.','Die Küche muss renoviert werden, sagte sie.',
                        'Bitte, mach mir das Leben nicht schwer.',
                        'Heute muss ich zum Zahnarzt, obwohl es Sonntag ist.','Wie viele Einheiten bilden ein Dutzend?',
                        'Was für einen Stift braucht man für das Dokument?','Hände hoch, oder die Hose runter.']

for sentence in test_sentences_german:
    print(f'In: {sentence} | Out: {translate(sentence)}')